<a href="https://colab.research.google.com/github/unburied/DS-Unit-1-Sprint-1-Dealing-With-Data/blob/master/LS_DS_111_A_First_Look_at_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - A First Look at Data



## Lecture - let's explore Python DS libraries and examples!

The Python Data Science ecosystem is huge. You've seen some of the big pieces - pandas, scikit-learn, matplotlib. What parts do you want to see more of?

I'm excited to learn about Theano and all of its high level children. Getting into neural networks feels like it would be the best end goal, especially with Tesla's announcement recently about its adavancements using adavanced neural networks. These are exciting times for this spectrum of data science and I am eager to learn more about it. 

## Assignment - now it's your turn

Pick at least one Python DS library, and using documentation/examples reproduce in this notebook something cool. It's OK if you don't fully understand it or get it 100% working, but do put in effort and look things up.

I hope this is ok. I actually worked on this a little after the precourse assignment was done. I feel like it demonstrates something cool and practical(ish) at the same time. 
https://colab.research.google.com/drive/1WMdcl9USQjuDE5n0JKwR0j9l-uqd7zXc


### Assignment questions

After you've worked on some code, answer the following questions in this text block:

1.  Describe in a paragraph of text what you did and why, as if you were writing an email to somebody interested but nontechnical.

In the project I linked above , I was able to make some loose predictions about the company that I currently work for. We receive monthly profit reports from the CEO. I was able to gather almost 2 years worth of data, and feed it to a linear regression model. Some of the cool libraries I was able to use and learn about were google.colab to import files from my computer, and datetime to convert the dates data to useable variables. This was on top of the skilearn library which houses the linear regression model I used to make the predictions. I wanted to see approximately when cd sales where going to become unprofitable, and when our growth product, books, would intersect and surpass it as a product to keep the company afloat. The linear regression model helps with this goal, as it takes the values I submitted, and creates a line of best fit from that data. You can then trace the line past the data you have to make predictions. At the current rate, cd sales and book sales intersect in October next year, and books will steadily climb into a profitable product while cd sales continue to decline significantly.  

2.  What was the most challenging part of what you did?

The challenging parts were importing the csv file I created in excel from my computer. It turns out colab can be a little tricky with that, and apprently a library was created to make this easier. Also, converting the dates to useable data was not trivial. I still need to gain a better understanding of what datetime.toordinal acutally does. 

3.  What was the most interesting thing you learned?

Besides the programming aspects of overcoming the above challenges, I learned first hand that the linear regression model is very limited. For instance, the prediction is that cd sales bottom out in 5 years. I find that unrealistic, and actually expect it will plateau at some point and hover well above the zero mark for at least the next decade. 

4.  What area would you like to explore with more time?

I would like to discover other models that could allow for the predicted data to plateau, or in a sense, output logarithmic regression as opposed to strictly linear.  Also, researching whats under the hood of datetime and toordinal. 




## Stretch goals and resources

Following are *optional* things for you to take a look at. Focus on the above assignment first, and make sure to commit and push your changes to GitHub (and since this is the first assignment of the sprint, open a PR as well).

- [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/)
- [scikit-learn documentation](http://scikit-learn.org/stable/documentation.html)
- [matplotlib documentation](https://matplotlib.org/contents.html)
- [Awesome Data Science](https://github.com/bulutyazilim/awesome-datascience) - a list of many types of DS resources

Stretch goals:

- Find and read blogs, walkthroughs, and other examples of people working through cool things with data science - and share with your classmates!
- Write a blog post (Medium is a popular place to publish) introducing yourself as somebody learning data science, and talking about what you've learned already and what you're excited to learn more about.

Awesome walkthrough on associatvie analysis:
https://pbpython.com/market-basket-analysis.html

My first blog post ever:
https://medium.com/@davi86m/conflicted-science-f767e94a1217
